# MOVIE RECOMMENDER SYSTEM 
###                           ~ by MAITRA TIWARI

In [ ]:
import numpy as np
import pandas as pd

In [70]:
credits=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')


In [71]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [72]:
movies.shape

(4803, 20)

In [73]:
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [74]:
credits.shape

(4803, 4)

In [75]:
movies = movies.merge(credits, on='title')

## Finding Usefull Columns for Recommender System.

In [76]:
movies['original_language'].value_counts().head(5)
# Since majoriy of movies have original language 'english',
# we can consider it as useless column.

original_language
en    4510
fr      70
es      32
zh      27
de      27
Name: count, dtype: int64

## Keeping Usefull Columns: 'genres', 'movie_id', 'keyword', 'title', 'overview', 'cast' and 'crew'

In [77]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [78]:
movies.head(3)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


## Data Cleaning

In [79]:
movies.isnull().sum() # Check missing values

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [80]:
movies.dropna(inplace=True)

In [81]:
movies.duplicated().sum() # Check Duplicated

0

## Converting 'genres' 'keywords' 'cast' 'crew' columns into Tags

In [82]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [83]:
import ast
def convert(obj):
    list = []
    for i in ast.literal_eval(obj):
        list.append(i['name'])
    return list

In [84]:
movies['genres'].apply(convert)

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4806, dtype: object

In [85]:
movies['keywords'].apply(convert)

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4806, dtype: object

In [86]:
movies['genres'] = movies['genres'].apply(convert)

In [87]:
movies['keywords'] = movies['keywords'].apply(convert)

In [88]:
movies.head(2) # 'generes' column changes done

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [89]:
movies.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [90]:
# Fetching top 3 actor's name.
import ast
def convert3(obj):
    list = []
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            list.append(i['name'])
            counter+=1
        else:
            break
    return list

In [91]:
movies['cast']=movies['cast'].apply(convert3) # Fetching name of top 3 actores of movie by their original name.

In [92]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [93]:
movies['crew'][0] # We are interested in fetching name of director.

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [94]:
# Fetching Director name from list of dictionary.
import ast
def director(obj):
    list = []
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            list.append(i['name'])
            break;
    return list

In [95]:
movies['crew']=movies['crew'].apply(director)

In [96]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [97]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [98]:
movies.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [99]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def clean_overview(words):
    words = [w.lower() for w in words if w.isalpha()]
    words = [ps.stem(w) for w in words if w not in stop_words]
    return " ".join(words)

movies['overview_clean'] = movies['overview'].apply(clean_overview)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maitr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Removing Spaces from the list entities of 'genres','keywords', 'cast' and 'crew' column to generate unique tags.

In [100]:
movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

0       [Action, Adventure, Fantasy, ScienceFiction]
1                       [Adventure, Fantasy, Action]
2                         [Action, Adventure, Crime]
3                   [Action, Crime, Drama, Thriller]
4                [Action, Adventure, ScienceFiction]
                            ...                     
4804                       [Action, Crime, Thriller]
4805                               [Comedy, Romance]
4806               [Comedy, Drama, Romance, TVMovie]
4807                                              []
4808                                   [Documentary]
Name: genres, Length: 4806, dtype: object

In [101]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [102]:
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [103]:
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [104]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [105]:
def collapse(list_of_words):
    return " ".join(list_of_words)

movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)  # assuming only director kept
movies['crew'] = movies['crew'].str.replace(" ", "")



## Creating a 'Tag' Column by merging 'overview', 'genres','keywords', 'cast' and 'crew' columns.

In [106]:
movies['tags'] = (
    movies['genres'] + " " +
    movies['keywords'] + " " +
    movies['overview_clean'] + " " +
    movies['crew']
)


In [107]:
movies.head(5)

,movie_id,title,overview,genres,keywords,cast,crew,overview_clean,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron,parapleg marin dispatch moon pandora uniqu bec...,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",GoreVerbinski,captain long believ come back life head edg ea...,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...",Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,"[DanielCraig, ChristophWaltz, LéaSeydoux]",SamMendes,cryptic messag past send trail uncov sinist ba...,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...",Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,"[ChristianBale, MichaelCaine, GaryOldman]",ChristopherNolan,follow death district attorney harvey batman a...,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,"[TaylorKitsch, LynnCollins, SamanthaMorton]",AndrewStanton,john carter former militari captain inexplic t...,Action Adventure ScienceFiction basedonnovel m...


In [108]:
movies.iloc[0]

movie_id                                                      19995
title                                                        Avatar
overview          [In, the, 22nd, century,, a, paraplegic, Marin...
genres                      Action Adventure Fantasy ScienceFiction
keywords          cultureclash future spacewar spacecolony socie...
cast                  [SamWorthington, ZoeSaldana, SigourneyWeaver]
crew                                                   JamesCameron
overview_clean    parapleg marin dispatch moon pandora uniqu bec...
tags              Action Adventure Fantasy ScienceFiction cultur...
Name: 0, dtype: object

In [116]:
df = movies[['movie_id','title','tags']]

In [117]:
df 
# New Cleaned and working dataframe

,movie_id,title,tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...
...,...,...,...
4804,9367,El Mariachi,Action Crime Thriller unitedstates–mexicobarri...
4805,72766,Newlyweds,Comedy Romance newlyw honeymoon upend arriv r...
4806,231617,"Signed, Sealed, Delivered",Comedy Drama Romance TVMovie date loveatfirsts...
4807,126186,Shanghai Calling,ambiti new york attorney sam sent shanghai i...


In [118]:
df['tags'].dtype

dtype('O')

In [119]:
df.head(5)

,movie_id,title,tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...


In [120]:
df['tags'] = df['tags'].apply(lambda x: x.lower())

C:\Users\maitr\AppData\Local\Temp\ipykernel_24740\4276611149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: x.lower())


In [121]:
df.head(5)

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


## Vectorization of 'Tags' Column.

In [122]:
df['tags'] = df['tags'].fillna('').astype(str)


C:\Users\maitr\AppData\Local\Temp\ipykernel_24740\1355487625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].fillna('').astype(str)


In [123]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [124]:
vectors = cv.fit_transform(df['tags']).toarray()

In [125]:
cv.fit_transform(df['tags']).toarray().shape

(4806, 5000)

In [126]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [127]:
features = cv.get_feature_names_out()


## Applying stemming (reducing words with the same root to a common base form)

In [128]:
!pip install nltk

In [129]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [130]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [131]:
ps.stem('loving')
ps.stem('lovely') # root word of both is 'love'

'love'

In [132]:
df['tags'].apply(stem) # every word replaced with it's root word.

0       action adventur fantasi sciencefict culturecla...
1       adventur fantasi action ocean drugabus exotici...
2       action adventur crime spi basedonnovel secreta...
3       action crime drama thriller dccomic crimefight...
4       action adventur sciencefict basedonnovel mar m...
                              ...                        
4804    action crime thriller unitedstates–mexicobarri...
4805    comedi romanc newlyw honeymoon upend arriv res...
4806    comedi drama romanc tvmovi date loveatfirstsig...
4807    ambiti new york attorney sam sent shanghai imm...
4808    documentari obsess camcord crush dreamgirl eve...
Name: tags, Length: 4806, dtype: object

In [133]:
df['tags'] = df['tags'].apply(stem)

C:\Users\maitr\AppData\Local\Temp\ipykernel_24740\866399325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [134]:
features = cv.get_feature_names_out()

In [135]:
# for i in features:
#   print (i)
# Now repetative tags are eliminated!

### Now we have 4806 (no.of.rows/data point), 5000(no.of.tags) dimensional vectors.

### Every movie is now a vector in 5000-Dimensional Space and we need to get distance of each movie from every other movie.

### Greater is the distance, lesser is the similarity!
### We will use Cosine distance, instead of Euclidean distance, as dimensionality is high.

In [136]:
from sklearn.metrics.pairwise import cosine_similarity

In [137]:
# Similarity is opposite of distance.

In [138]:
similarity = cosine_similarity(vectors)

In [139]:
similarity[0]
# This shows similarity if movie[0] with every other movie, including itself.

array([1.        , 0.09534626, 0.06963106, ..., 0.02624319, 0.        ,
       0.        ])

In [140]:
list(enumerate(similarity[0])) # To assign indexes.

[(0, 1.0000000000000002),
 (1, 0.09534625892455922),
 (2, 0.06963106238227915),
 (3, 0.08444006618414981),
 (4, 0.2229978693352242),
 (5, 0.14712247158412492),
 (6, 0.0),
 (7, 0.17407765595569785),
 (8, 0.07106690545187017),
 (9, 0.1144727614064285),
 (10, 0.12121212121212123),
 (11, 0.05802588531856595),
 (12, 0.10050378152592121),
 (13, 0.052486388108147805),
 (14, 0.15386436372416593),
 (15, 0.047822809985881044),
 (16, 0.09231861823449955),
 (17, 0.1462544848254261),
 (18, 0.10744306187005072),
 (19, 0.10618637464037979),
 (20, 0.08843153765707638),
 (21, 0.11677484162422844),
 (22, 0.08703882797784893),
 (23, 0.06963106238227915),
 (24, 0.0646508183835236),
 (25, 0.0652791209833867),
 (26, 0.14506471329641488),
 (27, 0.1969463855669324),
 (28, 0.1315903389919538),
 (29, 0.0787295821622217),
 (30, 0.08155908717405554),
 (31, 0.18148850216015697),
 (32, 0.10156734322380513),
 (33, 0.11396057645963797),
 (34, 0.0),
 (35, 0.11980845957463077),
 (36, 0.18181818181818185),
 (37, 0.09869

In [141]:
df[df.title=='Avatar'].index[0]  # To get index of particular movie.

0

In [142]:
def recommend(movie):
    movie_index = df[df.title==movie].index[0]
    distances=similarity[movie_index]
    # sorting the enumerated tuple in descending order, on the basis of similarity.
    re_movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    # printing the 5 recommended movie list.
    for i in re_movie_list:
        print(df.iloc[i[0]].title)

In [143]:
recommend('Avengers: Age of Ultron') # indexes of movie recommended for this movie.

Iron Man
The Wolverine
Iron Man 3
Captain America: Civil War
Iron Man 2


## Now for frontend/Streamlite

In [148]:
import pickle

In [149]:
df.title.values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [150]:
pickle.dump(df.to_dict(), open('movie_dict.pkl','wb'))

In [153]:
pickle.dump(similarity, open('Similarity.pkl','wb'))